## BernoulliNB

We are using the BernoulliNB algorithm form scikit-learn package (https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.BernoulliNB.html)in this notebook. We are solving the classification problem which we predict wether a PA form will be approved base on information provided on the PA form. Our data features are 'correct_diagnosis', 'tried_and_failed', 'contraindication', 'drug'(drug type), 'bin'(payer id),'reject_code', which are all categorical. Our label will be 'pa_approved'. 

In [1]:
#import pacakges
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

In [2]:
#read data
cmm_pa_clf_read = pd.read_csv("../Data/cmm_pa_clf.csv",index_col = 0)
cmm_pa_clf_data = cmm_pa_clf_read.drop(columns = 'pa_approved').copy()
cmm_pa_clf_target = cmm_pa_clf_read['pa_approved'].copy()
X_train,X_test,Y_train,Y_test= train_test_split(cmm_pa_clf_data, cmm_pa_clf_target, test_size = 0.2, 
                                             random_state = 10475, shuffle = True,
                                            stratify = cmm_pa_clf_target)

## Baseline:
We predoct that all PA form will be approved. In this case the true positive rate = false positive rate = 1, the ROC-AUC score of our baseline model is 0.5. The error of this predictor is 100-73.445 = 26.555.

## BernoulliNB  

The Bernoulli NB algorithm does not have much parameters, we will tune alpha (additive smoothing parameter). We will also compare the preformance of the algorithm on wether we learn the class prior or no.

In [3]:
tuned_parameters = {'alpha': [0.1*i for i in range(1,21)],'fit_prior': [True,False]}
scores = ['accuracy','roc_auc']
bnb_clf = BernoulliNB()
for scr in scores:
    print("# Tuning hyper-parameters for %s" % scr)
    print()
    clf_tun = GridSearchCV(bnb_clf, tuned_parameters, scoring="%s" % scr)
    clf_tun.fit(X_train, Y_train)
    print("Best parameters set found based on the parameter set:")
    print()
    print(clf_tun.best_params_)
    print("Grid scores on parameter set:")
    print()
    means = clf_tun.cv_results_["mean_test_score"]
    stds = clf_tun.cv_results_["std_test_score"]
    for mean, std, params in zip(means, stds, clf_tun.cv_results_["params"]):
        print("%0.3f (+/-%0.03f) for %r \n" % (mean, std * 2, params))
    print()

# Tuning hyper-parameters for accuracy

Best parameters set found based on the parameter set:

{'alpha': 0.1, 'fit_prior': True}
Grid scores on parameter set:

0.803 (+/-0.002) for {'alpha': 0.1, 'fit_prior': True} 

0.731 (+/-0.001) for {'alpha': 0.1, 'fit_prior': False} 

0.803 (+/-0.002) for {'alpha': 0.2, 'fit_prior': True} 

0.731 (+/-0.001) for {'alpha': 0.2, 'fit_prior': False} 

0.803 (+/-0.002) for {'alpha': 0.30000000000000004, 'fit_prior': True} 

0.731 (+/-0.001) for {'alpha': 0.30000000000000004, 'fit_prior': False} 

0.803 (+/-0.002) for {'alpha': 0.4, 'fit_prior': True} 

0.731 (+/-0.001) for {'alpha': 0.4, 'fit_prior': False} 

0.803 (+/-0.002) for {'alpha': 0.5, 'fit_prior': True} 

0.731 (+/-0.001) for {'alpha': 0.5, 'fit_prior': False} 

0.803 (+/-0.002) for {'alpha': 0.6000000000000001, 'fit_prior': True} 

0.731 (+/-0.001) for {'alpha': 0.6000000000000001, 'fit_prior': False} 

0.803 (+/-0.002) for {'alpha': 0.7000000000000001, 'fit_prior': True} 

0.731 (+/-0.001)

In [4]:
bnb_tuned = BernoulliNB(alpha = 0.1, fit_prior = True)
bnb_tuned.fit(X_train,Y_train)
Y_pred = bnb_tuned.predict(X_test)
print(classification_report(Y_test, Y_pred))
print('Accuacy score of this set of parameter is: ', accuracy_score(Y_test, Y_pred),'\n')
print('ROC-AUC score of this set of parameter is: ', roc_auc_score(Y_test, Y_pred),'\n')

              precision    recall  f1-score   support

           0       0.76      0.38      0.51     29527
           1       0.81      0.96      0.88     81664

    accuracy                           0.80    111191
   macro avg       0.79      0.67      0.69    111191
weighted avg       0.80      0.80      0.78    111191

Accuacy score of this set of parameter is:  0.8041747983200079 

ROC-AUC score of this set of parameter is:  0.6692764907408213 



In [5]:
bnb_tuned = BernoulliNB(alpha = 0.3, fit_prior = True)
bnb_tuned.fit(X_train,Y_train)
Y_pred = bnb_tuned.predict(X_test)
print(classification_report(Y_test, Y_pred))
print('Accuacy score of this set of parameter is: ', accuracy_score(Y_test, Y_pred),'\n')
print('ROC-AUC score of this set of parameter is: ', roc_auc_score(Y_test, Y_pred),'\n')

              precision    recall  f1-score   support

           0       0.76      0.38      0.51     29527
           1       0.81      0.96      0.88     81664

    accuracy                           0.80    111191
   macro avg       0.79      0.67      0.69    111191
weighted avg       0.80      0.80      0.78    111191

Accuacy score of this set of parameter is:  0.8041747983200079 

ROC-AUC score of this set of parameter is:  0.6692764907408213 

